In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import re
import math, os
from kaggle_datasets import KaggleDatasets
AUTO = tf.data.experimental.AUTOTUNE

print('Tensorflow Version {}'.format(tf.__version__))

# TPU or GPU Detection

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs

    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)

except ValueError: # detect GPUs
    
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

# Data Directory

In [ ]:
TFrec_selected = '512x512'
for dir_path, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if TFrec_selected in dir_path:
            print(os.path.join(dir_path, filename))

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

print(GCS_DS_PATH)


# Parameters

In [ ]:
IMAGE_SIZE = [512,512]
HEIGHT  =IMAGE_SIZE[0]
WIDTH = IMAGE_SIZE[1]
EPOCHS = 15
BATCH_SIZE = 16*strategy.num_replicas_in_sync
NUM_TRAIN_IMAGES = 12753 # how to calculate this using ro?
NUM_VAL_IMAGES = 3712
NUM_TEST_IMAGES = 7382

STEPS_PER_EPOCH = NUM_TRAIN_IMAGES//BATCH_SIZE
AUTO = tf.data.experimental.AUTOTUNE
TRAIN_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512/train/*.tfrec')
VAL_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512/val/*.tfrec')                                   
TEST_FILENAME = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-512x512/test/*.tfrec')                                 

In [ ]:
Classes = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'] 

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels =3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example,LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example["class"], tf.int32)
    return image, label

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id" : tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum


def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    # auto interleaves reads from multiple file
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset


def get_validation_dataset(filenames):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(filenames, ordered=True):
    dataset = load_dataset(filenames, labeled = False, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

Data Augmentation

Way to reduce overfitting by randomly altering the training images as they fir into the model for training, thus increasing the size of the train dataset.

In [ ]:
# Preprocessing layer for Data Augmentation

data_aug_layer = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1, fill_mode='constant'),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.5, seed=None),
    #tf.keras.layers.experimental.preprocessing.RandomZoom((0.2,0.3), fill_mode='nearest'),
    #tf.keras.layers.experimental.preprocessing.RandomFlip( mode="horizontal_and_vertical")
])

# Create ImageDataGenerator
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
rotation_range = 30, width_shift_range=0.15, height_shift_range=0.15,
brightness_range=None, zoom_range=[1.0,1.25], fill_mode='constant',
horizontal_flip=True, preprocessing_function=None)


def img_gen_random_transform(image, label):
    # apply random transform in single image
    image = img_gen.random_transform(image)
    return image, label


def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.9,0.99)
    image = tf.image.random_brightness(image, 0.5)
    image = tf.image.random_saturation(image, 0.8,0.9)
    return image, label

# Image Augmentation using tf.image

In [ ]:
# function to show data aug 

def show_data_aug(images):
    ROW = len(images)
    COL = 7
    plt.figure(figsize=(COL*2, ROW*2))
    i = 0
    for image in images:
        plt.subplot(ROW, COL, i*COL+1)
        plt.title("rd Flip L/R")
        plt.axis('off')
        plt.imshow(tf.image.random_flip_left_right(image))
        #plt.show()
        
        plt.subplot(ROW, COL, i*COL+2)
        plt.title("resize & rdm crop")
        plt.axis("off")
        image1 = tf.image.resize_with_crop_or_pad(image, HEIGHT+180,WIDTH+180)
        image1 = tf.image.random_crop(image1, size=[*IMAGE_SIZE,3])
        plt.imshow(image1)
        #plt.show()
        
        
        plt.subplot(ROW, COL, i*COL+3)
        plt.title("rdm contrast")
        plt.axis("off")
        plt.imshow(tf.image.random_contrast(image, 0.90,0.99))
        #plt.show()
        
        plt.subplot(ROW, COL, i*COL+4)
        plt.title("rndm brightness")
        plt.axis("off")
        plt.imshow(tf.image.random_brightness(image, 0.5))
        #plt.show()
        
        plt.subplot(ROW, COL, i*COL+5)
        plt.title("No Aug", color="r")
        plt.axis("off")
        plt.imshow(image)
        #plt.show()
        
        plt.subplot(ROW, COL, i*COL+6)
        plt.title("rdm saturation")
        plt.axis("off")
        plt.imshow(tf.image.random_saturation(image, 0.9,0.95))
        #plt.show()
        
        
        plt.subplot(ROW, COL, i*COL+7)
        plt.title("rdm hue")
        plt.axis("off")
        plt.imshow(tf.image.random_hue(image, 0.3))
        i+=1
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()  
        

# Training Dataset

In [ ]:
# trainign dataset with data augmentation
def get_training_dataset(filenames, augmentation=False):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    if augmentation:
        dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=1920)
    dataset = dataset.batch(BATCH_SIZE)
    
    if augmentation:
        dataset = dataset.map(lambda image, y: (data_aug_layer(image, training=True), y))
        #dataset = dataset.map(data_aug_layer)
        dataset = dataset.prefetch(AUTO)
    return dataset
    
    
#Get training dataset without any augmentation
no_aug_train_set = get_training_dataset(TRAIN_FILENAMES, augmentation=False)

print(no_aug_train_set)

# Visulaization of Images in Batch

In [ ]:
def batch_to_numpy_images_and_labels(databatch):
    images, labels = databatch
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    
    if numpy_labels.dtype == "O":
        numpy_labels = [None for _ in enumerate(numpy_images)]
    return numpy_images, numpy_labels

    
def show_images(databatch, row = 6, col = 8):
    FIGSIZE = (col*2, row*2)
    plt.figure(figsize=FIGSIZE)
    images, num_label = batch_to_numpy_images_and_labels(databatch)
    for j in range(row*col):
        plt.subplot(row, col, j+1)
        plt.axis('off')
        plt.title(Classes[num_label[j]])
        plt.imshow(images[j])
    plt.show()


In [ ]:
no_aug_train_batch = next(iter(no_aug_train_set.unbatch().batch(16)))
# print(no_aug_train_batch)                    
images, _ = batch_to_numpy_images_and_labels(no_aug_train_batch)

print("Image Augmentation in Train using tf.image")
show_data_aug(images)

# Image Augmentaton: ImageDataGenerator

In [ ]:
img_dgen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=36,
                                                          width_shift_range = 20,
                                                          height_shift_range=0.2,
                                                          horizontal_flip=True,
                                                          fill_mode = "constant",
                                                          brightness_range=None,
                                                          zoom_range =[0.75, 1.0],
                                                          preprocessing_function = None)

print("Training Dataset")
print("Image  Augmentation by using random transformation using Image Data Generator")
i = 0
ROW = 8
COL = 4
plt.figure(figsize=(COL*3, ROW*3.5))
for image in images:
    plt.subplot(ROW, COL, i*2+1)
    plt.title("No Aug")
    plt.axis("off")
    plt.imshow(image)
    
    plt.subplot(ROW, COL, i*2+2)
    plt.title('rndm trns from img d_gene')
    plt.axis("off")
    plt.imshow(img_dgen.random_transform(image))
    i = i+1
plt.show()    

# Image Augmentation: Keras Layers

In [ ]:
print("Image Augmentation using tf.keras.preprocessing")

ROW = len(images)
COL = 4 
plt.figure(figsize=(COL*3, ROW*3))
i = 0

for image in images:
    
    plt.subplot(ROW, COL, i*4+1)
    plt.title("No Aug", color='red')
    plt.imshow(image)
    plt.axis("off")
    
    plt.subplot(ROW, COL, i*4+2)
    plt.title("Rndm Zoom")
    img = tf.keras.preprocessing.image.random_zoom(image, (-0.3,-0.2),  fill_mode="reflect")
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(ROW, COL, i*4+3)
    plt.title("Rndm Shear")
    img = tf.keras.preprocessing.image.random_shear(image,20, fill_mode = "nearest")
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(ROW, COL, i*4+4)
    plt.title("Rndm Rotate")
    img = tf.keras.preprocessing.image.random_rotation(image,20, fill_mode = "nearest")
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(ROW, COL, i*4+4)
    plt.title("Rndm Brightness")
    img = tf.keras.preprocessing.image.random_brightness(image, (0.02,0.035))
    
    plt.imshow(img)
    plt.axis("off")
    
    i += 1 
plt.show()

# Setting Train, Test and Validation dataset

In [ ]:
training_dataset = get_training_dataset(TRAIN_FILENAMES, augmentation=True)
validation_dataset = get_validation_dataset(VAL_FILENAMES)
test_dataset = get_test_dataset(TEST_FILENAME)


print("Training dataset {}".format(training_dataset))
print("Validation dataset {}".format(validation_dataset))
print("Test dataset {}".format(test_dataset))

In [ ]:
# display training dataset

R = 10
C = 10
B = R*C  #batch size

print("Displaying Training Dataset with Keras Layers Augmentation")
show_images(next(iter(training_dataset.unbatch().batch(B))), row=R, col=C)

In [ ]:
# displaying validation dataset
show_images(next(iter(validation_dataset.unbatch().batch(B))), row = R, col = C)

# Model

In [ ]:
#This callback will stop the training when there is no improvement in the validation loss for the 3 consecutive epochs.
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3)
BEST_MODEL_PATH = "/kaggle/working/model_best.h5"
FILE_DIR = os.path.dirname(BEST_MODEL_PATH)

# create a checkpoint to save the best trained model locally
save_locally = tf.saved_model.SaveOptions(experimental_io_device = '/job:localhost')
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = BEST_MODEL_PATH,
                                                options = save_locally, monitor = 'val_loss', verbose=1, 
                                                save_best_only = True, save_weights_only =False, mode='min')

In [ ]:
with strategy.scope():
    pretrained_base_model = tf.keras.applications.DenseNet121 (weights = 'imagenet',
                                                          include_top = False,
                                                          input_shape = [*IMAGE_SIZE, 3])
    for layer in pretrained_base_model.layers:
        layer.trainable = False
    
    model = tf.keras.Sequential([
        pretrained_base_model,
        #tf.keras.layers.Flatten(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(len(Classes), activation = 'softmax', kernel_regularizer = tf.keras.regularizers.L2(0.00011))
    ])

    model.summary()
    
    model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy']
    )
    
    history = model.fit(
    training_dataset,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_dataset,
    callbacks = [es_callback, cp_callback]

    )

# Plot Accuracy and Loss curve

In [ ]:
history.history.keys()

In [ ]:
epoch_range = range(1, len(history.history['loss'])+1)
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history ['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(25,5))
plt.subplot(1,2,1)
plt.plot(epoch_range, acc, label = "Training Accuracy")
plt.plot(epoch_range, val_acc, label = "Validation Accuracy")
plt.legend(loc='best')
plt.title("Training and Validation Accuracy", size=15)
plt.xlabel('Epochs')

plt.subplot(1,2,2)
plt.plot(epoch_range, loss, label = "Training Loss")
plt.plot(epoch_range, val_loss, label = "Validation Loss")
plt.legend(loc='best')
plt.title("Training and Validation Loss", size=15)
plt.xlabel('Epochs')


In [ ]:
print("List of al entries in {}: {}".format(FILE_DIR, tf.io.gfile.listdir(FILE_DIR)))
best_model = tf.keras.models.load_model(BEST_MODEL_PATH)

In [ ]:
print("Computing predictions")
test_dataset_ds = test_dataset.map(lambda image, idnum: image)
probabilities = model.predict(test_dataset_ds)

prediction = np.argmax(probabilities, axis=-1)
print("Predictions {}".format(prediction))

In [ ]:
print("Generating submission file")
test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

submission_df = pd.DataFrame({'id': test_ids, 'label': prediction })
submission_df.to_csv('submission.csv', index = False)

print('Submitting output...... ')
#np.savetxt('submission.csv', np.rec.fromarrays([test_ids,prediction]), fmt = ['%s', '%d'], delimiter = ',', header = 'id, label')